# Offspect collimated reduction

Trying to reproduce https://docs.mantidproject.org/nightly/algorithms/ReflectometryReductionOne-v2.html

In [ ]:
import scipp as sc
import scippneutron as scn
import plopp as pp
from ess import amor, reflectometry
import numpy as np
import ess

pp.patch_scipp()
%matplotlib widget

In [ ]:
logger = ess.logging.configure_workflow('offspec_reduction',
                                        filename=None)

## Load data

In [ ]:
sample = scn.load("../../../../data/OFFSPEC00062107.nxs", mantid_args={"LoadMonitors": True})

In [ ]:
direct_beam = scn.load("../../../../data/OFFSPEC00062163.nxs", mantid_args={"LoadMonitors": True},)

In [ ]:
sample

In [ ]:
direct_beam

TODO use actual metadata

In [ ]:
from orsopy import fileio
from ess.amor.orso import make_orso

owner = fileio.base.Person('Jochen Stahn', 'Paul Scherrer Institut', 'jochen.stahn@psi.ch')
creator = fileio.base.Person('Andrew R. McCluskey', 'European Spallation Source', 'andrew.mccluskey@ess.eu')

orso = make_orso(owner=owner,
                 sample=fileio.data_source.Sample('Ni/Ti Multilayer', 'gas/solid', 'air | (Ni | Ti) * 5 | Si'),
                 creator=creator,
                 reduction_script='https://github.com/scipp/ess/blob/main/docs/instruments/amor/amor_reduction.ipynb')

In [ ]:
sample.attrs['orso'] = sc.scalar(orso)
direct_beam.attrs['orso'] = sc.scalar(orso)

In [ ]:
sample.hist(tof=200).plot()

In [ ]:
direct_beam.hist(tof=200).sum('spectrum').plot()

sample and direct_beam are misaligned in spectrum.

In [ ]:
s = sample.hist(tof=200)
d = direct_beam.hist(tof=s.coords['tof'])
r = s / d
r.plot()

## Convert to wavelength

In [ ]:
graph = {**reflectometry.conversions.specular_reflection()}

In [ ]:
wavelength_edges = sc.linspace('wavelength', 1, 14.0, 1000, unit='Å')
w_sample = reflectometry.conversions.tof_to_wavelength(sample, wavelength_edges)

In [ ]:
w_sample.hist().plot()

Sum all spectra in direct beam (like Mantid).
Summing (`bins.concat`) removes the position coord. Since the direct beam should be detected at theta=0, we use this to define the position. `specular_pixel` finds the spectrum index for the specular peak.

In [ ]:
def specular_pixel(da):
    dims = list(da.dims)
    del dims[dims.index('spectrum')]
    dim, = dims
    return np.argmax(da.sum(dim).values) + da.coords['spectrum'].min().value

In [ ]:
no_scatter_graph = {**scn.conversion.graph.beamline.beamline(scatter=False),
                     **scn.conversion.graph.tof.elastic_wavelength(start='tof')}

In [ ]:
da = direct_beam.bins.concat('spectrum')
da.coords['position'] = direct_beam.coords['position'][specular_pixel(direct_beam)]
w_direct_beam = da.transform_coords('wavelength', no_scatter_graph)

## Normalise by monitor

Mantid does not normalise the direct beam by monitor.

In [ ]:
sample_mon = sample.attrs['monitor2'].value
w_sample_mon = sample_mon.transform_coords('wavelength', graph=no_scatter_graph)
direct_beam_mon = direct_beam.attrs['monitor2'].value
w_direct_beam_mon = direct_beam_mon.transform_coords('wavelength', graph=no_scatter_graph)

Drop variances.
The alpha value seems to be low enough.

In [ ]:
w_sample_mon = sc.values(w_sample_mon)
w_direct_beam_mon = sc.values(w_direct_beam_mon)

In [ ]:
w_sample_norm_mon = w_sample.bins / sc.lookup(w_sample_mon, dim='wavelength')
w_direct_beam_norm_mon = w_direct_beam.bins / sc.lookup(w_direct_beam_mon, dim='wavelength')

## Normalise by direct beam

Align sample and direct_beam by cropping both in spectrum around the specular peak.
Then pretend that the direct beam was measured at the same pixels ('spectrum' and 'position') as the sample.

In [ ]:
# sample_specular_pixel = np.argmax(w_sample_norm_mon.sum('wavelength').values) + w_sample_norm_mon.coords['spectrum'].min().value
# direct_beam_specular_pixel = np.argmax(w_direct_beam.sum('wavelength').values) + w_direct_beam.coords['spectrum'].min().value
# width = 15
# cropped_sample = w_sample_norm_mon['spectrum', sc.index(sample_specular_pixel-width): sc.index(sample_specular_pixel+width)].copy()
# cropped_direct_beam = w_direct_beam['spectrum', sc.index(direct_beam_specular_pixel-width): sc.index(direct_beam_specular_pixel+width)].copy()
# cropped_direct_beam.coords['spectrum'] = cropped_sample.coords['spectrum'].copy()
# cropped_direct_beam.attrs['position'] = cropped_sample.attrs['position'].copy()

In [ ]:
# using CROPPED
# ref = sc.values(cropped_direct_beam).hist()
# w_norm = cropped_sample / ref
# w_norm.masks['no_reference_neutrons'] = (ref == sc.scalar(0, unit='count')).data
# w_norm.coords['sample_rotation'] = sample.attrs['Theta'].value[-1].data

In [ ]:
ref = sc.values(w_direct_beam_norm_mon).hist(wavelength=w_sample.coords['wavelength'])
# ??? w_norm has no position even with this del
# del ref.attrs['position']
w_norm = w_sample / ref
w_norm.masks['no_reference_neutrons'] = (ref == sc.scalar(0, unit='one')).data
w_norm.coords['sample_rotation'] = sample.attrs['Theta'].value[-1].data
w_norm.attrs['position'] = w_sample.attrs['position']
w_norm.attrs['orso'] = w_sample.attrs['orso']

In [ ]:
width = 15
crop_w_norm = w_sample_norm_mon['spectrum', sc.index(404-width): sc.index(404+width)].copy()

In [ ]:
crop_w_norm.hist().plot(norm='log')

In [ ]:
crop_w_norm.hist().sum('spectrum').plot(norm='log')

In [ ]:
x = crop_w_norm.copy(deep=False)
x.attrs['gravity'] = sc.vector([0, -1, 0]) * sc.constants.g
x.coords['sample_rotation'] = sample.attrs['Theta'].values[-1].data
theta_norm = reflectometry.conversions.wavelength_to_theta(x, graph=graph)

WHY ARE THESE NEGATIVE?

In [ ]:
theta_norm.coords['theta']

In [ ]:
theta_norm.hist().sum('spectrum').plot(norm='log')

## Convert to Q

Gravity seems to be in -y direction.

In [ ]:
norm_q = theta_norm.transform_coords(['Q'], graph=graph)

In [ ]:
norm_q

In [ ]:
norm_q.hist().plot(norm='log')

In [ ]:
norm_q.hist().sum('spectrum').plot(norm='log')

# Compare with Mantid

## I vs tof

showing a normalised histogram to compare with mantid's plots

In [ ]:
da = direct_beam.bins.concat('spectrum').hist(tof=200)
assert sc.islinspace(da.coords['tof'], 'tof').value
x = sc.midpoints(da.coords['tof']).values
y = da.data.values / (x[1]-x[0])
import matplotlib.pyplot as plt
f = plt.figure()
ax = f.add_subplot(111)
ax.plot(x, y)

## i vs wavelength

In [ ]:
w_sample.bins.concat('spectrum').hist().plot()

In [ ]:
w_direct_beam.hist(wavelength=1000).plot()

## sample normalised by direct beam

In [ ]:
x = w_sample / sc.values(w_direct_beam.hist(wavelength=wavelength_edges))
x.bins.concat('spectrum').hist().plot()